# Background 
PowerModelsProtection is based on PowerModels for the transmission system and PowerModelsDistribution for distribution system. The focus has been on the distribution system because of its use in Smart-Gird applications so the distribution formulation is more developed than the transmission formulation. Currently, the distribution solve is being rewritten to focus more on matrix formulations rather than on individual constraints (discussion later). 

jtabarez@lanl.gov

LA-UR-25-20118
# Installing PowerModelsProtection.jl
Using the Julia terminal and the command "add PowerModelProtection"

The formulation is described in *Optimization-Based Formulations for Short-Circuit Studies with Inverter-Interfaced Generation in PowerModelsProtection.jl*
The focus of PMP is on faults with inverters. 

# Short-Circuit Constraints
The base equation for faults is based on the current injection into the fault
$$I^{\phi}_{si_ff} = {\bm G}_fV^{\phi}_{i_f}$$
To develop the constraint the $${\bm G}_f$$ matrix needs to be defined for each type of fault. 

The faults that PMP can perform:
- Single line to ground (most common of faults in distribution system)
- Line to Line 
- Line to Line to ground 
- 3 Phase 
- 3 Phase to ground 

# Math Models 
Single line to ground
$${\bm G}_f = 
\begin{bmatrix}
\mathbf{g}_f & 0 & 0\\
0 & 0 & 0 \\
0 & 0 & 0\\
\end{bmatrix}$$

Line to line is seen as a connected load between phases. 
$${\bm G}_f = 
\begin{bmatrix}
\mathbf{g}_f & -\mathbf{g}_f & 0\\
-\mathbf{g}_f & \mathbf{g}_f & 0 \\
0 & 0 & 0\\
\end{bmatrix}$$

For three phase to ground faults, the connection is seen as fault connections to a common point which will be used when the fault includes a connection to ground. This avoids having the neutral being explicitly modeled throughout the problem or could be modeled as a simple delta impedance load because that is what the math model becomes. The star-mesh transformation in required for the ground fault and is used here to model the unground three phase fault. 

![title](img/3_phase.png) 
 
The matrix equation is:
$${\bm G}_f =3
\begin{bmatrix}
2\mathbf{g}_f & -\mathbf{g}_f & -\mathbf{g}_f\\
-\mathbf{g}_f & 2\mathbf{g}_f & -\mathbf{g}_f \\
-\mathbf{g}_f & -\mathbf{g}_f & 2\mathbf{g}_f\\
\end{bmatrix}$$

For line to line to ground faults the star-mesh transformation is used to avoid the issue of neutral discussed above. 

![title](img/llg.png) 

Following the transformation: 
$$\mathbf{g}_{total} = 2\mathbf{g}_{p}+\mathbf{g}_{g}$$

$$\mathbf{g}_{pp} = \frac{\mathbf{g}_{p}^2}{\mathbf{g}_{total}}$$

$$\mathbf{g}_{pg} = \frac{\mathbf{g}_{p}\mathbf{g}_{g}}{\mathbf{g}_{total}}$$

The matrix is then: 
$${\bm G}_f = 
\begin{bmatrix}
\mathbf{g}_{pp}\mathbf{g}_{pg} & -\mathbf{g}_{pp} & 0\\
-\mathbf{g}_{pp} & \mathbf{g}_{pp}\mathbf{g}_{pg} & 0 \\
0 & 0 & 0\\
\end{bmatrix}$$

For the three phase to ground fault

![title](img/3_phase_g.png)

Following the transformation:

$$\mathbf{g}_{total} = 3\mathbf{g}_{p}+\mathbf{g}_{g}$$

$$\mathbf{g}_{pp} = \frac{\mathbf{g}_{p}^2}{\mathbf{g}_{total}}$$

$$\mathbf{g}_{pg} = \frac{\mathbf{g}_{p}\mathbf{g}_{g}}{\mathbf{g}_{total}}$$

The matrix is then:
$${\bm G}_f = 
\begin{bmatrix}
2\mathbf{g}_{pp}\mathbf{g}_{pg} & -\mathbf{g}_{pp} & -\mathbf{g}_{pp}\\
-\mathbf{g}_{pp} & 2\mathbf{g}_{pp}\mathbf{g}_{pg} & -\mathbf{g}_{pp} \\
-\mathbf{g}_{pp} & -\mathbf{g}_{pp} & 2\mathbf{g}_{pp}\mathbf{g}_{pg}\\
\end{bmatrix}$$

In the end, any fault can be modeled just as long it is defined as current injection, because the coupling constraint of the problem is KCL. KCL couples the nodes by the branch injections. 

For non-faulted nodes: 
$$\sum_{(i,j,k) \in {\cal E}_i^+}I^{\phi}_{ijkf} - \sum_{(j,i,k) \in {\cal E}_i^-}I^{\phi}_{ijkf} = \sum_{g \in \mathrm{G}(i)}I^{\phi}_{gf} - \sum_{l \in \mathrm{L}(i)}I^{\phi}_{lf}$$

For faulted nodes:
$$\sum_{(i,j,k) \in {\cal E}_{i_f}^+}I^{\phi}_{ijkf} - \sum_{(j,i,k) \in {\cal E}_{i_f}^-}I^{\phi}_{ijkf} = \sum_{g \in \mathrm{G}(i_f)}I^{\phi}_{gf} - \sum_{l \in \mathrm{L}(i)}I^{\phi}_{lf} - I^\phi_{i_ff}$$

Then it is just power flow. 


## Running PMP
What is shown in this section is based on v0.5.3 but the current version of PMP is at v0.7.1 (will talk about later).

Like most faults solvers PMP will perform a fault study, meaning it will perform all faults on all nodes. It is important to realize that there can be infeasible solutions or numerical errors because the formulation is non-convex and non-linear. Most will argue that short-circuits problems are linear which is true when power defined loads and shunts, and any other non-linear components like inverters or induction motors are not modeled. To reduce model complexity loads and shunts can be dropped, which most commercial solvers have an option to include or exclude them but even without loads transformer connections can cause the admittance to be singular (or close to) causing solvers to crash or reach their max iterations.  


In [13]:
using PowerModelsProtection

import Ipopt
import PowerModelsDistribution

const PMD = PowerModelsDistribution

PowerModelsDistribution.silence!()

ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-6, "print_level"=>0)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("tol") => 1.0e-6, MathOptInterface.RawOptimizerAttribute("print_level") => 0])

OpenDSS format for distribution systems is currently the only format supported. To load case: 

In [14]:
case =  parse_file("C:\\Users\\338983\\Documents\\GitHub\\Winter_School\\dss\\3bus_example.dss")

Dict{String, Any} with 12 entries:
  "conductor_ids"  => [1, 2, 3, 4]
  "method"         => "PMD"
  "bus"            => Dict{String, Any}("primary"=>Dict{String, Any}("rg"=>Floa…
  "name"           => "3bus_example"
  "settings"       => Dict{String, Any}("sbase_default"=>500.0, "vbases_default…
  "files"          => ["C:\\Users\\338983\\Documents\\GitHub\\Winter_School\\ds…
  "dss_options"    => Dict{String, Any}("voltagebases"=>[0.4], "defaultbasefreq…
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("connection…
  "line"           => Dict{String, Any}("quad"=>Dict{String, Any}("length"=>1.0…
  "data_model"     => ENGINEERING
  "load"           => Dict{String, Any}("l2"=>Dict{String, Any}("model"=>POWER,…
  "linecode"       => Dict{String, Any}("556mcm"=>Dict{String, Any}("b_fr"=>[25…

To perform a fault study: 

In [15]:
fault_studies = build_mc_fault_study(case)
sol = solve_mc_fault_study(case, fault_studies, ipopt_solver)

Dict{String, Any} with 3 entries:
  "primary"  => Dict{String, Any}("3p"=>Dict{String, Any}("1"=>Dict{String, Any…
  "primary2" => Dict{String, Any}("3p"=>Dict{String, Any}("1"=>Dict{String, Any…
  "loadbus"  => Dict{String, Any}("3p"=>Dict{String, Any}("1"=>Dict{String, Any…

The layout of the results will be:

___Node (string int)

________Fault type ("3p", "3pg", "ll", "llg", "lg")

_________________Fault number ("1", "2", "3") these are based on the number of phases or fault (ll -- "1" = a to b)

_____________________________"Solution" (Contains the normal solution information including whether the problem was feasible, solution time, etc.. )




In [16]:
sol

Dict{String, Any} with 3 entries:
  "primary"  => Dict{String, Any}("3p"=>Dict{String, Any}("1"=>Dict{String, Any…
  "primary2" => Dict{String, Any}("3p"=>Dict{String, Any}("1"=>Dict{String, Any…
  "loadbus"  => Dict{String, Any}("3p"=>Dict{String, Any}("1"=>Dict{String, Any…

In [17]:
sol["primary"]






Dict{String, Any} with 5 entries:
  "3p"  => Dict{String, Any}("1"=>Dict{String, Any}("solve_time"=>0.0170002, "o…
  "ll"  => Dict{String, Any}("1"=>Dict{String, Any}("solve_time"=>0.016, "optim…
  "lg"  => Dict{String, Any}("1"=>Dict{String, Any}("solve_time"=>0.00699997, "…
  "3pg" => Dict{String, Any}("1"=>Dict{String, Any}("solve_time"=>0.00999999, "…
  "llg" => Dict{String, Any}("1"=>Dict{String, Any}("solve_time"=>0.013, "optim…

In [18]:
sol["primary"]["ll"]

Dict{String, Any} with 3 entries:
  "1" => Dict{String, Any}("solve_time"=>0.016, "optimizer"=>"Ipopt", "terminat…
  "2" => Dict{String, Any}("solve_time"=>0.013, "optimizer"=>"Ipopt", "terminat…
  "3" => Dict{String, Any}("solve_time"=>0.00999999, "optimizer"=>"Ipopt", "ter…

In [19]:
sol["primary"]["ll"]["1"]

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.016
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("voltage_source"=>Dict{String, Any}…
  "objective_lb"       => -Inf

To perform a single fault, a fault is added to the case data: 

In [20]:
add_fault!(case, "1", "3p", "loadbus", [1,2,3], 0.005)
sol = solve_mc_fault_study(case, ipopt_solver)
println(sol["solution"]["fault"]["1"]["cf"][1])

1040.7458422515847


Issues with PMP:
- Slow 
- Inverters require binaries


## Inverter Modeling 
Inverter's current saturates:

![title](img/inverter_model_curve.png)

To model this non-linear behavior, a binary would have to be used to indicate that the current injection is constant when voltage on the node collapses. Work has been carried out to relax the model, but there are cases that are not feasible. 

The follwoing is the grid-following results for line to line: 

![title](img/Relaxed_following.png)

The grid-forming relaxations have some issues: 
![title](img/grid_forming.png)


# New Formulations in v0.7.1 (Currently under development)
Back to $$I=YV$$

This will follow the new format of JuMP of taking in matrices based problems. 

The admittance matrix is explicitly defined (me being a power engineer), and non-linear components are linearized into the matrix with non-linear adjustments made in the vector of the current. This allows access for individual primitive matrices for each component to be adjusted or viewed by the user. I think this is helpful when dealing with transformer configurations. 

Currently the only solver is an iterative based solver, but work has started to convert back to the optimization-based solver in which the main linear constraint will be I=YV and all non-linear constraints will be applied to I vector. 

Inverter fault standards (positive and negative sequence currents) are being incorporated into the solver. 

Show IEEE123 if there is time. 

In [21]:
data = parse_file("C:\\Users\\338983\\Documents\\GitHub\\Winter_School\\dss\\Modified IEEE123 DSS files\\IEEE123MasterV2_Relays_RONM_no_pv.dss")

model = PowerModelsProtection.instantiate_mc_admittance_model(data;loading=true) 



AdmittanceModel(Dict{String, Any}("is_kron_reduced" => false, "conductor_ids" => [1, 2, 3, 4], "time_elapsed" => 1.0, "bus" => Dict{String, Any}("32" => Dict{String, Any}("vm_pair_lb" => Tuple{Any, Any, Real}[], "grounded" => Bool[0, 0], "vm_pair_ub" => Tuple{Any, Any, Real}[], "bus_i" => 32, "name" => "26", "bus_type" => 1, "terminals" => [1, 3], "vmax" => [Inf, Inf], "vbase" => 2.402, "source_id" => "bus.26"…), "29" => Dict{String, Any}("vm_pair_lb" => Tuple{Any, Any, Real}[], "grounded" => Bool[0, 1], "vm_pair_ub" => Tuple{Any, Any, Real}[], "bus_i" => 29, "name" => "92", "bus_type" => 1, "terminals" => [3, 4], "vmax" => [Inf, Inf], "vbase" => 2.401777119828844, "source_id" => "bus.92"…), "1" => Dict{String, Any}("vm_pair_lb" => Tuple{Any, Any, Real}[], "grounded" => Bool[0, 1], "vm_pair_ub" => Tuple{Any, Any, Real}[], "bus_i" => 1, "name" => "32", "bus_type" => 1, "terminals" => [3, 4], "vmax" => [Inf, Inf], "vbase" => 2.402, "source_id" => "bus.32"…), "54" => Dict{String, Any}("vm

In [22]:
model.data

Dict{String, Any} with 21 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("32"=>Dict{String, Any}("vm_pair_lb"=>…
  "name"            => "ieee123"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>100000.0, "loading"=>…
  "controls"        => Dict{String, Any}()
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("32"=>Dict{String, Any}("br_r"=>[0.081…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}("8"=>Dict{String, Any}("f_connections"…
  "admittance_type" => Dict{Int64, Any}(56=>1, 35=>1, 60=>1, 220=>1, 308=>1, 67…
  "is_projected"    => false
  "per_unit"        => false
  "data_model"      => MATHEMATICAL
  "admittance_map"  => Dict{Tuple, Int64}((111, 2)=>50, (141, 3)=>59, (125, 2)=…
  "shunt"   

In [23]:
model.data["transformer"]["1"]

Dict{String, Any} with 30 entries:
  "polarity"      => [1, 1]
  "cm_ub"         => Inf
  "sm_nom"        => [5000.0, 5000.0]
  "tm_lb"         => [[0.9, 0.9, 0.9], [0.9, 0.9, 0.9]]
  "tm_set"        => [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
  "tm_step"       => [[0.03125, 0.03125, 0.03125], [0.03125, 0.03125, 0.03125]]
  "f_connections" => [1, 2, 3, 4]
  "configuration" => ConnConfig[WYE, WYE]
  "name"          => "reg1a"
  "b_sh"          => -0.0
  "element"       => Transformer2WElement
  "tm_nom"        => [4.16, 4.16]
  "status"        => 1
  "g_sh"          => 0.0
  "dss"           => Dict{String, Any}("windings"=>2, "name"=>"reg1a", "phases"…
  "xsc"           => [1.0e-5]
  "source_id"     => "transformer.reg1a"
  "t_connections" => [1, 2, 3, 4]
  "f_bus"         => 61
  ⋮               => ⋮

In [24]:
model.data["transformer"]["1"]["p_matrix"]

8×8 Matrix{ComplexF64}:
  288.895-28889.5im       0.0+0.0im      …   288.895-28889.5im
      0.0+0.0im       288.895-28889.5im      288.895-28889.5im
      0.0+0.0im           0.0+0.0im          288.895-28889.5im
 -288.895+28889.5im  -288.895+28889.5im     -866.685+86668.5im
 -288.895+28889.5im       0.0+0.0im         -288.895+28889.5im
      0.0+0.0im      -288.895+28889.5im  …  -288.895+28889.5im
      0.0+0.0im           0.0+0.0im         -288.895+28889.5im
  288.895-28889.5im   288.895-28889.5im      866.685-86668.5im